In [ ]:
import pandas as pd
import os
import glob
import numpy as np

# Compare Function

In [ ]:
def compare(df, comp1=None, comp2=None, comp3=None):
    if df[comp1] == df[comp2]:
        return "Match"
    else:
        return "No Match"

# Smartsheets

In [ ]:
path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type1 = ".xlsx"
load1 = glob.glob(path1 + file_type1)
df1 = pd.read_excel(max(load1, key=os.path.getctime), parse_dates=True, keep_default_na=False)

### Vehicle Removal and total setup

In [ ]:
vecs = df1.copy()
vecs = vecs[['APN_ROW Segment','Structural Status', 'Street #', 'Street Name','County',
             'Total Number of Vehicles']]

In [ ]:
vecs.groupby(['County'])['Total Number of Vehicles'].sum()

# RTR for vehicle removal

In [ ]:
path2 = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
file_type2 = ".xlsx"
load2 = glob.glob(path2 + file_type2)
df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True, na_values=None)

In [ ]:
RTR = df2.copy()
RTR = RTR[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2']]
RTR = RTR.copy()
RTR['Zone Name'] = RTR['Zone Name'].str[:3] + "-" + RTR['Zone Name'].str[3:6] + "-" + RTR['Zone Name'].str[6:]
RTR['Service Code'] = RTR['Service Code'].str.strip()

In [ ]:
RTR_Vecs = RTR.copy()
RTR_Vecs = RTR_Vecs[(RTR_Vecs['Is Void'] == False) & (RTR_Vecs['Service Code'].isin(['5A','5B']))]

In [ ]:
# RTR_Vecs['GIS Zone2'].

In [ ]:
RTR_Vecs = RTR_Vecs[RTR_Vecs['GIS Zone1'].isin(['CENTRAL DIVISION'])]

In [ ]:
RTR_Vecs = RTR_Vecs[['Zone Name', 'Unit Count', 'GIS Zone2']]

In [ ]:
RTR_SUM_Vecs = RTR_Vecs.groupby(['Zone Name'])[['Unit Count']].sum().reset_index()

In [ ]:
RTR_SUM_Vecs = RTR_SUM_Vecs.add_suffix("_RTR")

### Merge smartsheets and rtr total vehicles

In [ ]:
RTR_vec_final = vecs.merge(RTR_SUM_Vecs,
                             left_on='APN_ROW Segment',
                             right_on='Zone Name_RTR',
                             how='outer')

In [ ]:
RTR_vec_final.head(1)

In [ ]:
RTR_vec_final['Unit Count_RTR'] = RTR_vec_final['Unit Count_RTR'].replace(np.nan, 0)
RTR_vec_final['Total Number of Vehicles'] = RTR_vec_final['Total Number of Vehicles'].fillna(0)

In [ ]:
RTR_vec_final.insert(RTR_vec_final.columns.get_loc('Total Number of Vehicles')+1, "Match",
                                    RTR_vec_final.apply(compare, comp1='Total Number of Vehicles',
                                                        comp2='Unit Count_RTR', axis=1))

In [ ]:
RTR_vec_final_style = RTR_vec_final.style.set_properties(**{'background-color': 'lightblue'},
                                        subset=['Zone Name_RTR', 'Unit Count_RTR'])

In [ ]:
RTR_vec_final_style.to_excel("RTR Total vehicle removed check.xlsx")

In [ ]:
RTR_vec_final.head(1)

In [ ]:
RTR_vec_final.groupby(['County'])['Total Number of Vehicles'].sum()

In [ ]:
RTR_vec_final.groupby(['County'])['Unit Count_RTR'].sum()